<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-&amp;-Tools" data-toc-modified-id="Import-&amp;-Tools-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import &amp; Tools</a></span></li><li><span><a href="#Repnet-Model" data-toc-modified-id="Repnet-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Repnet Model</a></span></li><li><span><a href="#Load-Model" data-toc-modified-id="Load-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Model</a></span><ul class="toc-item"><li><span><a href="#Modify-Add-OP-Shape-Align(Dim-Tile)" data-toc-modified-id="Modify-Add-OP-Shape-Align(Dim-Tile)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Modify Add OP Shape Align(Dim Tile)</a></span></li><li><span><a href="#Simple-Conv3D-Model" data-toc-modified-id="Simple-Conv3D-Model-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Simple Conv3D Model</a></span></li></ul></li><li><span><a href="#Frozen-Graph" data-toc-modified-id="Frozen-Graph-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Frozen Graph</a></span></li><li><span><a href="#Saved-Model" data-toc-modified-id="Saved-Model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Saved Model</a></span><ul class="toc-item"><li><span><a href="#Modify-Transpose-Perm-Attributes(FusedBatchNormV3)" data-toc-modified-id="Modify-Transpose-Perm-Attributes(FusedBatchNormV3)-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Modify Transpose Perm Attributes(FusedBatchNormV3)</a></span></li><li><span><a href="#Original-Model-Outputs-VS-Modified-Model-Outputs" data-toc-modified-id="Original-Model-Outputs-VS-Modified-Model-Outputs-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Original Model Outputs VS Modified Model Outputs</a></span></li><li><span><a href="#Onnx-Optimizer" data-toc-modified-id="Onnx-Optimizer-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Onnx Optimizer</a></span></li><li><span><a href="#Repnet-Inference" data-toc-modified-id="Repnet-Inference-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Repnet Inference</a></span></li><li><span><a href="#Calculate-Count" data-toc-modified-id="Calculate-Count-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Calculate Count</a></span><ul class="toc-item"><li><span><a href="#Inv-Period-Confidence-Score" data-toc-modified-id="Inv-Period-Confidence-Score-5.5.1"><span class="toc-item-num">5.5.1&nbsp;&nbsp;</span>Inv-Period Confidence Score</a></span></li><li><span><a href="#Within-Period-Confience-Scores" data-toc-modified-id="Within-Period-Confience-Scores-5.5.2"><span class="toc-item-num">5.5.2&nbsp;&nbsp;</span>Within Period Confience Scores</a></span></li><li><span><a href="#Period-Count" data-toc-modified-id="Period-Count-5.5.3"><span class="toc-item-num">5.5.3&nbsp;&nbsp;</span>Period Count</a></span></li></ul></li></ul></li><li><span><a href="#TFlite" data-toc-modified-id="TFlite-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>TFlite</a></span></li><li><span><a href="#References" data-toc-modified-id="References-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>References</a></span></li><li><span><a href="#FAQs" data-toc-modified-id="FAQs-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>FAQs</a></span></li></ul></div>

## Import & Tools

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p cv2,PIL,matplotlib
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard
%watermark -p onnx,onnxruntime,tf2onnx
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np
import copy

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass


numpy 1.19.5
sklearn 0.24.2
pandas 1.1.5
cv2 4.5.3
PIL 8.2.0
matplotlib 3.3.4
torch 1.9.0
torchvision 0.10.0
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0
onnx 1.10.1
onnxruntime 1.8.1
tf2onnx 1.9.2


In [2]:
def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<center><img %s %s src="%s"/></center>' % (W, H, data_url))


In [3]:
def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<center><video %s %s controls src="%s" type="video/mp4"/></center>' % (W, H, data_url))



In [4]:
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'tensorboard-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))

@register_line_cell_magic
def template_writefile(line, cell):
    with open(line, 'w') as fw:
        fw.write(cell.format(**globals()))

@register_line_magic
def netron(line):
    args = line.split()
    file, port, height = args[0], int(args[1]), 600
    if len(args) == 3:
        height = int(args[2])
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    try:
        netron.start(file, address=('0.0.0.0', port), browse=False)
        display_html(port, height)
    except:
        pass

In [5]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow.lite.python.util import run_graph_optimizations, get_grappler_config
import onnx
import onnx.helper as OH
import onnxruntime as rt
from onnxsim import simplify

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
gpus

[]

## Repnet Model

Repnet模型转换为onnx进行推理时, 存在两个问题:

<div class="alert alert-warning">
    <li> 加法算子操作输入shape不一致问题(pos_encoding)</li>
    <li> 3D卷积(5-D)后进行BatchNormlization时, 不支持5D</li>
</div>

In [6]:
layers = tf.keras.layers
regularizers = tf.keras.regularizers


def get_sims(embs, temperature):
    """Calculates self-similarity between batch of sequence of embeddings."""
    batch_size = tf.shape(embs)[0]
    seq_len = tf.shape(embs)[1]
    embs = tf.reshape(embs, [batch_size, seq_len, -1])

    def _get_sims(embs):
        """Calculates self-similarity between sequence of embeddings."""
        dist = pairwise_l2_distance(embs, embs)
        sims = -1.0 * dist
        return sims

    sims = tf.map_fn(_get_sims, embs)
    sims /= temperature
    sims = tf.nn.softmax(sims, axis=-1)
    sims = tf.expand_dims(sims, -1)
    return sims


def flatten_sequential_feats(x, batch_size, seq_len):
    """Flattens sequential features with known batch size and seq_len."""
    x = tf.reshape(x, [batch_size, seq_len, -1])
    return x


# Transformer from https://www.tensorflow.org/tutorials/text/transformer .
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.

      q, k, v must have matching leading dimensions.
      k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
      The mask has different shapes depending on its type(padding or look ahead)
      but it must be broadcastable for addition.

      Args:
        q: query shape == (..., seq_len_q, depth)
        k: key shape == (..., seq_len_k, depth)
        v: value shape == (..., seq_len_v, depth_v)
        mask: Float tensor with shape broadcastable
              to (..., seq_len_q, seq_len_k). Defaults to None.

      Returns:
        outputs: shape == (..., seq_len_q, depth_v)
        attention_weights: shape == (..., seq_len_q, seq_len_k)
      """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk.
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    # (..., seq_len_q, seq_len_k)
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    outputs = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return outputs, attention_weights


def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])


class MultiHeadAttention(tf.keras.layers.Layer):
    """Multi-headed attention layer."""

    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth)."""
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(
            q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(
            k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(
            v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(
            scaled_attention,
            perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(
            scaled_attention,
            (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        outputs = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return outputs, attention_weights


class TransformerLayer(tf.keras.layers.Layer):
    """Implements a single transformer layer (https://arxiv.org/abs/1706.03762).
    """

    def __init__(self, d_model, num_heads, dff,
                 dropout_rate=0.1,
                 reorder_ln=False):
        super(TransformerLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

        self.reorder_ln = reorder_ln

    def call(self, x):
        inp_x = x

        if self.reorder_ln:
            x = self.layernorm1(x)

        # (batch_size, input_seq_len, d_model)
        attn_output, _ = self.mha(x, x, x, mask=None)
        attn_output = self.dropout1(attn_output)

        if self.reorder_ln:
            out1 = inp_x + attn_output
            x = out1
        else:
            # (batch_size, input_seq_len, d_model)
            out1 = self.layernorm1(x + attn_output)
            x = out1

        if self.reorder_ln:
            x = self.layernorm2(x)

        # (batch_size, input_seq_len, d_model)
        ffn_output = self.ffn(x)
        ffn_output = self.dropout2(ffn_output)

        if self.reorder_ln:
            out2 = out1 + ffn_output
        else:
            # (batch_size, input_seq_len, d_model)
            out2 = self.layernorm2(out1 + ffn_output)

        return out2


def pairwise_l2_distance(a, b):
    """Computes pairwise distances between all rows of a and all rows of b."""
    norm_a = tf.reduce_sum(tf.square(a), 1)
    norm_a = tf.reshape(norm_a, [-1, 1])
    norm_b = tf.reduce_sum(tf.square(b), 1)
    norm_b = tf.reshape(norm_b, [1, -1])
    dist = tf.maximum(norm_a - 2.0 * tf.matmul(a, b, False, True) + norm_b, 0.0)
    return dist


class ResnetPeriodEstimator(tf.keras.models.Model):
    """RepNet model."""

    def __init__(
        self,
        num_frames=64,
        image_size=112,
        base_model_layer_name='conv4_block3_out',
        temperature=13.544,
        dropout_rate=0.25,
        l2_reg_weight=1e-6,
        temporal_conv_channels=512,
        temporal_conv_kernel_size=3,
        temporal_conv_dilation_rate=3,
        conv_channels=32,
        conv_kernel_size=3,
        transformer_layers_config=((512, 4, 512),),
        transformer_dropout_rate=0.0,
        transformer_reorder_ln=True,
        period_fc_channels=(512, 512),
        within_period_fc_channels=(512, 512)):
        super(ResnetPeriodEstimator, self).__init__()

        # Model params.
        self.num_frames = num_frames
        self.image_size = image_size

        self.base_model_layer_name = base_model_layer_name

        self.temperature = temperature

        self.dropout_rate = dropout_rate
        self.l2_reg_weight = l2_reg_weight

        self.temporal_conv_channels = temporal_conv_channels
        self.temporal_conv_kernel_size = temporal_conv_kernel_size
        self.temporal_conv_dilation_rate = temporal_conv_dilation_rate

        self.conv_channels = conv_channels
        self.conv_kernel_size = conv_kernel_size
        # Transformer config in form of (channels, heads, bottleneck channels).
        self.transformer_layers_config = transformer_layers_config
        self.transformer_dropout_rate = transformer_dropout_rate
        self.transformer_reorder_ln = transformer_reorder_ln

        self.period_fc_channels = period_fc_channels
        self.within_period_fc_channels = within_period_fc_channels

        # Base ResNet50 Model.
        base_model = tf.keras.applications.ResNet50V2(
            include_top=False, weights=None, pooling='max')
        self.base_model = tf.keras.models.Model(
            inputs=base_model.input,
            outputs=base_model.get_layer(self.base_model_layer_name).output)

        # 3D Conv on k Frames
        self.temporal_conv_layers = [
            layers.Conv3D(self.temporal_conv_channels,
                          self.temporal_conv_kernel_size,
                          padding='same',
                          dilation_rate=(self.temporal_conv_dilation_rate, 1, 1),
                          kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                          kernel_initializer='he_normal')]
        self.temporal_bn_layers = [layers.BatchNormalization(axis=4)
                                   for _ in self.temporal_conv_layers]

        # Counting Module (Self-sim > Conv > Transformer > Classifier)
        self.conv_3x3_layer = layers.Conv2D(self.conv_channels,
                                            self.conv_kernel_size,
                                            padding='same',
                                            activation=tf.nn.relu)

        channels = self.transformer_layers_config[0][0]
        self.input_projection = layers.Dense(
            channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
            activation=None)

        self.input_projection2 = layers.Dense(
            channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
            activation=None)

        length = self.num_frames
        self.pos_encoding = tf.compat.v1.get_variable(
            name='resnet_period_estimator/pos_encoding',
            shape=[1, length, 1],
            initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))
        self.pos_encoding2 = tf.compat.v1.get_variable(
            name='resnet_period_estimator/pos_encoding2',
            shape=[1, length, 1],
            initializer=tf.compat.v1.truncated_normal_initializer(stddev=0.02))

        self.transformer_layers = []
        for d_model, num_heads, dff in self.transformer_layers_config:
            self.transformer_layers.append(
                TransformerLayer(d_model, num_heads, dff,
                                 self.transformer_dropout_rate,
                                 self.transformer_reorder_ln))

        self.transformer_layers2 = []
        for d_model, num_heads, dff in self.transformer_layers_config:
            self.transformer_layers2.append(
                TransformerLayer(d_model, num_heads, dff,
                                 self.transformer_dropout_rate,
                                 self.transformer_reorder_ln))

        # Period Prediction Module.
        self.dropout_layer = layers.Dropout(self.dropout_rate)
        num_preds = self.num_frames//2
        self.fc_layers = []
        for channels in self.period_fc_channels:
            self.fc_layers.append(layers.Dense(
                channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                activation=tf.nn.relu))
        self.fc_layers.append(layers.Dense(
            num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

        # Within Period Module
        num_preds = 1
        self.within_period_fc_layers = []
        for channels in self.within_period_fc_channels:
            self.within_period_fc_layers.append(layers.Dense(
                channels, kernel_regularizer=regularizers.l2(self.l2_reg_weight),
                activation=tf.nn.relu))
        self.within_period_fc_layers.append(layers.Dense(
            num_preds, kernel_regularizer=regularizers.l2(self.l2_reg_weight)))

    def call(self, x):
        # Ensures we are always using the right batch_size during train/eval.
        batch_size = tf.shape(x)[0]
        # Conv Feature Extractor.
        x = tf.reshape(x, [-1, self.image_size, self.image_size, 3])
        x = self.base_model(x)
        h = tf.shape(x)[1]
        w = tf.shape(x)[2]
        c = tf.shape(x)[3]
        x = tf.reshape(x, [batch_size, -1, h, w, c])

        # 3D Conv to give temporal context to per-frame embeddings. 
        for bn_layer, conv_layer in zip(self.temporal_bn_layers,
                                        self.temporal_conv_layers):
            x = conv_layer(x)
            x = bn_layer(x)
            x = tf.nn.relu(x)
            
        x = tf.reshape(x, (batch_size, self.num_frames, 7, 7, -1))
    
        x = tf.reduce_max(x, [2, 3])

        # Reshape and prepare embs for output.
        final_embs = x

        # Get self-similarity matrix.
        x = get_sims(x, self.temperature)

        # 3x3 conv layer on self-similarity matrix.
        x = self.conv_3x3_layer(x)
        x = tf.reshape(x, [batch_size, self.num_frames, -1])
        within_period_x = x

        # Period prediction.
        x = self.input_projection(x)
        x += self.pos_encoding
        for transformer_layer in self.transformer_layers:
            x = transformer_layer(x)
        x = flatten_sequential_feats(x, batch_size, self.num_frames)
        for fc_layer in self.fc_layers:
            x = self.dropout_layer(x)
            x = fc_layer(x)

        # Within period prediction.
        within_period_x = self.input_projection2(within_period_x)
        within_period_x += self.pos_encoding2
        for transformer_layer in self.transformer_layers2:
            within_period_x = transformer_layer(within_period_x)
        within_period_x = flatten_sequential_feats(within_period_x,
                                                   batch_size,
                                                   self.num_frames)
        for fc_layer in self.within_period_fc_layers:
            within_period_x = self.dropout_layer(within_period_x)
            within_period_x = fc_layer(within_period_x)

        return x, within_period_x, final_embs

## Load Model

### Modify Add OP Shape Align(Dim Tile)

In [7]:
model = ResnetPeriodEstimator()
print(model.pos_encoding.shape)
print(model.pos_encoding[0][0][:10])
ckpt = tf.train.Checkpoint(model=model)
ckpt.restore('/data/pretrained/cv/repnet/ckpt-88').expect_partial()
print('-'*20)
model.pos_encoding = tf.tile(model.pos_encoding, multiples=[1, 1, 512])
model.pos_encoding2 = tf.tile(model.pos_encoding2, multiples=[1, 1, 512])
print(model.pos_encoding.shape)
print(model.pos_encoding[0][0][:10])
test_inputs = np.random.randn(2, 64, 112, 112, 3).astype(np.float32)
test_inputs_tensor = tf.convert_to_tensor(test_inputs)
test_outputs = model(test_inputs_tensor)

(1, 64, 1)
tf.Tensor([-0.03162105], shape=(1,), dtype=float32)
--------------------
(1, 64, 512)
tf.Tensor(
[0.02054866 0.02054866 0.02054866 0.02054866 0.02054866 0.02054866
 0.02054866 0.02054866 0.02054866 0.02054866], shape=(10,), dtype=float32)


In [8]:
test_outputs[0].shape, test_outputs[1].shape, test_outputs[2].shape

(TensorShape([2, 64, 32]), TensorShape([2, 64, 1]), TensorShape([2, 64, 512]))

### Simple Conv3D Model

tool | version
---: | :---:
tensorflow | 2.6.0
tensorboard | 2.6.0
onnx | 1.10.1
onnxruntime | 1.8.1
tf2onnx | 1.9.2

<div class="alert alert-warning">
针对上面版本, 转换onnx格式时, BatchNormalization会和Transpose操作融合为FusedBatchNormV3, Transpose的dim参数让人疑惑
</div>

In [9]:
model3d = tf.keras.models.Sequential([
    layers.Conv3D(
        512, 3, padding='same',
        dilation_rate=(3, 1, 1),
        kernel_regularizer=regularizers.l2(1e-6),
        kernel_initializer='he_normal'),
    layers.BatchNormalization(axis=1),
    layers.ReLU()
])

inputs3d = np.random.randn(2, 64, 7, 7, 1024).astype(np.float32)
outputs3d = model3d(inputs3d)

In [10]:
saved_model3d_path = '/data/nb_data/saved_models3d'
model3d.save(saved_model3d_path, save_format='tf', include_optimizer=True)

INFO:tensorflow:Assets written to: /data/nb_data/saved_models3d/assets


In [11]:
!python3 -m tf2onnx.convert --opset 14 \
    --tag serve --signature_def serving_default \
    --saved-model {saved_model3d_path} --output {saved_model3d_path}/saved_model.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-09-02 21:55:35,210 - INFO - Signatures found in model: [serving_default].
2021-09-02 21:55:35,225 - INFO - Output names: ['re_lu']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-09-02 21:55:39,261 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:703: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-09-02 21:55:42,393 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.9.2/0f28b7
2021-09-02 21:55:42,393 - INFO - Using opset <onnx, 14>
2021-09-02 21:55:58,543 - INFO - Computed 0 values for constant folding
2021-09-02 21:56:09,509 - 

In [12]:
%netron {saved_model3d_path}/saved_model.onnx 8131 400

Serving '/data/nb_data/saved_models3d/saved_model.onnx' at http://0.0.0.0:8131


In [13]:
sess = rt.InferenceSession(f'{saved_model3d_path}/saved_model.onnx')  
inputs = {sess.get_inputs()[0].name: inputs3d}
outputs = [o.name for o in sess.get_outputs()]
predictions = sess.run(outputs, inputs)
predictions[-1].shape, outputs3d[-1].shape

((2, 64, 7, 7, 512), TensorShape([64, 7, 7, 512]))

## Frozen Graph

In [14]:
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(test_inputs.shape, test_inputs.dtype))
frozen_func = convert_variables_to_constants_v2(full_model, lower_control_flow=True)

In [15]:
[ x for x in dir(frozen_func) if x[0] != '_'], '-' * 60, \
frozen_func.inputs, frozen_func.outputs 

(['add_gradient_functions_to_graph',
  'add_to_graph',
  'captured_inputs',
  'function_def',
  'graph',
  'inputs',
  'name',
  'output_dtypes',
  'output_shapes',
  'outputs',
  'pretty_printed_signature',
  'prune',
  'structured_input_signature',
  'structured_outputs',
  'trainable_variables',
  'variables'],
 '------------------------------------------------------------',
 [<tf.Tensor 'x:0' shape=(2, 64, 112, 112, 3) dtype=float32>],
 [<tf.Tensor 'Identity:0' shape=(2, 64, 32) dtype=float32>,
  <tf.Tensor 'Identity_1:0' shape=(2, 64, 1) dtype=float32>,
  <tf.Tensor 'Identity_2:0' shape=(2, 64, 512) dtype=float32>])

In [16]:
[ x for x in dir(frozen_func.graph) if x[0] != '_'], '-' * 60, \
frozen_func.graph.inputs, frozen_func.graph.outputs

(['add_capture',
  'add_to_collection',
  'add_to_collections',
  'as_default',
  'as_graph_def',
  'as_graph_element',
  'building_function',
  'capture',
  'capture_by_value',
  'capture_call_time_value',
  'capture_distributed_variable',
  'capture_eager_tensor',
  'captured',
  'captures',
  'clear_captures',
  'clear_collection',
  'collections',
  'colocate_with',
  'container',
  'control_captures',
  'control_dependencies',
  'control_outputs',
  'create_op',
  'deferred_external_captures',
  'deferred_internal_captures',
  'device',
  'external_captures',
  'finalize',
  'finalized',
  'get_all_collection_keys',
  'get_collection',
  'get_collection_ref',
  'get_name_scope',
  'get_operation_by_name',
  'get_operations',
  'get_tensor_by_name',
  'gradient_override_map',
  'graph_def_versions',
  'inputs',
  'internal_captures',
  'is_control_flow_graph',
  'is_feedable',
  'is_fetchable',
  'mark_as_unsaveable',
  'name',
  'name_scope',
  'outer_graph',
  'output_shapes',
  

In [17]:
frozen_graph_def = frozen_func.graph.as_graph_def()
[ x for x in dir(frozen_graph_def) if x[0] != '_']

['ByteSize',
 'Clear',
 'ClearExtension',
 'ClearField',
 'CopyFrom',
 'DESCRIPTOR',
 'DiscardUnknownFields',
 'Extensions',
 'FindInitializationErrors',
 'FromString',
 'HasExtension',
 'HasField',
 'IsInitialized',
 'ListFields',
 'MergeFrom',
 'MergeFromString',
 'ParseFromString',
 'RegisterExtension',
 'SerializePartialToString',
 'SerializeToString',
 'SetInParent',
 'UnknownFields',
 'WhichOneof',
 'library',
 'node',
 'version',
 'versions']

In [18]:
operations = frozen_func.graph.get_operations()
[ x for x in dir(operations[590]) if x[0] != '_']

['colocation_groups',
 'control_inputs',
 'device',
 'get_attr',
 'graph',
 'inputs',
 'name',
 'node_def',
 'op_def',
 'outputs',
 'run',
 'traceback',
 'type',
 'values']

In [19]:
for i, op in enumerate(operations[580: 595], 580):
    print('%5d' % i, op.name)

  580 resnet_period_estimator/Reshape_1/shape/1
  581 resnet_period_estimator/Shape_1
  582 resnet_period_estimator/strided_slice_1/stack
  583 resnet_period_estimator/strided_slice_1/stack_1
  584 resnet_period_estimator/strided_slice_1/stack_2
  585 resnet_period_estimator/strided_slice_1
  586 resnet_period_estimator/Shape_2
  587 resnet_period_estimator/strided_slice_2/stack
  588 resnet_period_estimator/strided_slice_2/stack_1
  589 resnet_period_estimator/strided_slice_2/stack_2
  590 resnet_period_estimator/strided_slice_2
  591 resnet_period_estimator/Shape_3
  592 resnet_period_estimator/strided_slice_3/stack
  593 resnet_period_estimator/strided_slice_3/stack_1
  594 resnet_period_estimator/strided_slice_3/stack_2


In [20]:
print(operations[590])

name: "resnet_period_estimator/strided_slice_2"
op: "StridedSlice"
input: "resnet_period_estimator/Shape_2"
input: "resnet_period_estimator/strided_slice_2/stack"
input: "resnet_period_estimator/strided_slice_2/stack_1"
input: "resnet_period_estimator/strided_slice_2/stack_2"
attr {
  key: "Index"
  value {
    type: DT_INT32
  }
}
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}
attr {
  key: "begin_mask"
  value {
    i: 0
  }
}
attr {
  key: "ellipsis_mask"
  value {
    i: 0
  }
}
attr {
  key: "end_mask"
  value {
    i: 0
  }
}
attr {
  key: "new_axis_mask"
  value {
    i: 0
  }
}
attr {
  key: "shrink_axis_mask"
  value {
    i: 1
  }
}



In [21]:
# input_tensors = [x for x in frozen_func.inputs if x.dtype != tf.resource]
# output_tensors = frozen_func.outputs
# 
# config = [
#     'pruning', 'function', 'constfold', 'shape', 'remap', 'memory',
#     'common_subgraph_elimination', 'arithmetic', 'loop', 'dependency', 'debug_stripper'
# ]
# 
# frozen_graph_def = run_graph_optimizations(
#     frozen_graph_def,
#     input_tensors,
#     output_tensors,
#     config=get_grappler_config(config),
#     graph=frozen_func.graph
# )

In [22]:
as_text = False
frozen_graph_proto_path = tf.io.write_graph(
    graph_or_graph_def=frozen_graph_def,
    logdir='/data/nb_data/frozen_models',
    name='frozen_graph.pb%s' % ('txt' if as_text else ''),
    as_text=as_text)

In [23]:
# %netron {frozen_graph_proto_path} 8120 500

In [24]:
inputs = ','.join([x.name for x in frozen_func.inputs])
outputs = ','.join([x.name for x in frozen_func.outputs])

!python3 -m tf2onnx.convert --opset 14 --input {frozen_graph_proto_path} \
    --inputs {inputs} --outputs {outputs} --output {frozen_graph_proto_path}.onnx

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
2021-09-02 21:56:41,023 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:305: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-09-02 21:56:41,023 - WARNING - From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/convert_to_constants.py:927: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instru

In [25]:
%netron {frozen_graph_proto_path}.onnx 8132 500

Serving '/data/nb_data/frozen_models/frozen_graph.pb.onnx' at http://0.0.0.0:8132


In [26]:
onnx_model = onnx.load(f'{frozen_graph_proto_path}.onnx')
try:
    onnx.checker.check_model(onnx_model, full_check=True) 
except Exception as err:
    print(err)
graph_def = onnx_model.graph
[x for x in dir(graph_def) if x[0] != '_'], '-'*60, graph_def.input, '-'*60, graph_def.output

No Op registered for TensorListReserve with domain_version of 14

==> Context: Bad node spec for node. Name: resnet_period_estimator/map/TensorArrayV2_1 OpType: TensorListReserve


(['ByteSize',
  'Clear',
  'ClearExtension',
  'ClearField',
  'CopyFrom',
  'DESCRIPTOR',
  'DiscardUnknownFields',
  'Extensions',
  'FindInitializationErrors',
  'FromString',
  'HasExtension',
  'HasField',
  'IsInitialized',
  'ListFields',
  'MergeFrom',
  'MergeFromString',
  'ParseFromString',
  'RegisterExtension',
  'SerializePartialToString',
  'SerializeToString',
  'SetInParent',
  'UnknownFields',
  'WhichOneof',
  'doc_string',
  'initializer',
  'input',
  'name',
  'node',
  'output',
  'quantization_annotation',
  'sparse_initializer',
  'value_info'],
 '------------------------------------------------------------',
 [name: "x:0"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 2
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 3
       }
     }
   }
 }
 ],
 '------------------------------------------

In [27]:
for i, node in enumerate(graph_def.node[95:99], 95):
    print('%5d' % i, node)

   95 input: "resnet_period_estimator/Reshape_1:0"
output: "resnet_period_estimator/conv3d/Conv3D__174:0"
name: "resnet_period_estimator/conv3d/Conv3D__174"
op_type: "Transpose"
attribute {
  name: "perm"
  ints: 0
  ints: 4
  ints: 1
  ints: 2
  ints: 3
  type: INTS
}

   96 input: "resnet_period_estimator/conv3d/Conv3D__174:0"
input: "resnet_period_estimator/conv3d/Conv3D/ReadVariableOp:0"
input: "resnet_period_estimator/conv3d/BiasAdd/ReadVariableOp:0"
output: "Conv__281:0"
name: "Conv__281"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 3
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 3
  ints: 3
  ints: 3
  type: INTS
}
attribute {
  name: "pads"
  ints: 3
  ints: 1
  ints: 1
  ints: 3
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
domain: ""

   97 input: "Conv__281:0"
output: "resnet_period_estimator/batch_normalization/FusedBatc

## Saved Model

In [28]:
saved_model_path = '/data/nb_data/saved_models'
model.save(saved_model_path, save_format='tf', include_optimizer=True)

INFO:tensorflow:Assets written to: /data/nb_data/saved_models/assets


INFO:tensorflow:Assets written to: /data/nb_data/saved_models/assets
/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [29]:
!python3 -m tf2onnx.convert --opset 14 \
    --tag serve --signature_def serving_default \
    --saved-model {saved_model_path} --output {saved_model_path}/saved_model.onnx 

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-09-02 21:59:51,242 - INFO - Signatures found in model: [serving_default].
2021-09-02 21:59:51,249 - INFO - Output names: ['output_1', 'output_2', 'output_3']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-09-02 22:00:03,294 - WARNING - From /usr/local/lib/python3.6/dist-packages/tf2onnx/tf_loader.py:703: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-09-02 22:00:10,764 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.9.2/0f28b7
2021-09-02 22:00:10,764 - INFO - Using opset <onnx, 14>
2021-09-02 22:00:27,294 - INFO - Computed 0 values for constant folding

In [30]:
%netron {saved_model_path}/saved_model.onnx 8133 500

Serving '/data/nb_data/saved_models/saved_model.onnx' at http://0.0.0.0:8133


### Modify Transpose Perm Attributes(FusedBatchNormV3)

In [91]:
onnx_model = onnx.load(f'{saved_model_path}/saved_model.onnx')
onnx_model = onnx.shape_inference.infer_shapes(onnx_model, check_type=True) # for value_info
onnx_graph_def = onnx_model.graph
for i, node in enumerate(onnx_graph_def.node[180:195], 180):
    print('%5d' % i, node.name)
    if 'FusedBatchNormV3' in node.name and 'Transpose' == node.op_type:
        attr = node.attribute.pop()
        if attr.ints[1] == 3:
            node.attribute.append(OH.make_attribute('perm', [0, 4, 1, 2, 3]))
        elif attr.ints[1] == 2:
            node.attribute.append(OH.make_attribute('perm', [0, 2, 3, 4, 1]))
        print('-'*60)
        print(node)
        print('-'*60)
        
onnx.save(onnx_model, f'{saved_model_path}/saved_model_mod.onnx')

  180 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND_transpose__549
  181 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND_reshape__550
  182 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND_concat__551
  183 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND_gather__553
  184 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND_reshape__554
  185 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND_transpose__559
  186 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND_reshape__561
  187 StatefulPartitionedCall/resnet_period_estimator/conv3d/Conv3D/BatchToSpaceND
  188 StatefulPartitionedCall/resnet_period_estimator/conv3d/BiasAdd
  189 StatefulPartitionedCall/resnet_period_estimator/batch_normalization/FusedBatchNormV3__576
------------------------------------------------------------
input: "StatefulP

In [32]:
len(onnx_graph_def.node), len(onnx_graph_def.value_info), len(onnx_graph_def.initializer)

(480, 479, 206)

In [33]:
target_idx = 189
onnx_graph_def.value_info[target_idx], '-'*80, onnx_graph_def.node[target_idx]

(name: "StatefulPartitionedCall/resnet_period_estimator/batch_normalization/FusedBatchNormV3__576:0"
 type {
   tensor_type {
     elem_type: 1
   }
 },
 '--------------------------------------------------------------------------------',
 input: "StatefulPartitionedCall/resnet_period_estimator/conv3d/BiasAdd:0"
 output: "StatefulPartitionedCall/resnet_period_estimator/batch_normalization/FusedBatchNormV3__576:0"
 name: "StatefulPartitionedCall/resnet_period_estimator/batch_normalization/FusedBatchNormV3__576"
 op_type: "Transpose"
 attribute {
   name: "perm"
   ints: 0
   ints: 4
   ints: 1
   ints: 2
   ints: 3
   type: INTS
 })

In [34]:
for i, w in enumerate(onnx_graph_def.initializer):
    if onnx_graph_def.node[target_idx].input[0][:-2] in w.name:
        print(w.dims, w.name)

[512] StatefulPartitionedCall/resnet_period_estimator/conv3d/BiasAdd/ReadVariableOp:0


In [35]:
%netron {saved_model_path}/saved_model_mod.onnx 8134 400

Serving '/data/nb_data/saved_models/saved_model_mod.onnx' at http://0.0.0.0:8134


### Original Model Outputs VS Modified Model Outputs

In [36]:
sess = rt.InferenceSession(f'{saved_model_path}/saved_model_mod.onnx')  
inputs = {sess.get_inputs()[0].name: test_inputs}
outputs = [o.name for o in sess.get_outputs()]
outputs_onnx = sess.run(outputs, inputs)
outputs_onnx[-1].shape

(2, 64, 512)

In [69]:
 test_inputs.shape, sess.get_inputs()[0].name

((2, 64, 112, 112, 3), 'input_1')

In [37]:
test_outputs[0][0][0][:3].numpy(), '-'*60, outputs_onnx[0][0][0][:3]

(array([-13.030266 ,   3.4435723,   2.5435479], dtype=float32),
 '------------------------------------------------------------',
 array([-13.030245 ,   3.4435954,   2.5433795], dtype=float32))

In [38]:
test_outputs[1][0][:3].numpy().T, '-'*60, outputs_onnx[1][0][:3].T

(array([[0.84029174, 0.56367415, 0.97805107]], dtype=float32),
 '------------------------------------------------------------',
 array([[0.8402951, 0.5636726, 0.9780331]], dtype=float32))

### Onnx Optimizer

In [93]:
onnx_model = onnx.load(f'{saved_model_path}/saved_model_mod.onnx')  
model_simp, check = simplify(
    onnx_model, dynamic_input_shape=False,
    input_shapes={sess.get_inputs()[0].name: [1, 64, 112, 112, 3]})
if not check:
    raise
model_simp.graph.input, model_simp.graph.output

([name: "input_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 112
       }
       dim {
         dim_value: 3
       }
     }
   }
 }
 ],
 [name: "output_1"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 32
       }
     }
   }
 }
 , name: "output_2"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 1
       }
     }
   }
 }
 , name: "output_3"
 type {
   tensor_type {
     elem_type: 1
     shape {
       dim {
         dim_value: 1
       }
       dim {
         dim_value: 64
       }
       dim {
         dim_value: 512
       }
     }
   }
 }
 ])

In [94]:
onnx.save(model_simp, f'{saved_model_path}/saved_model_simp.onnx')

In [84]:
%netron {saved_model_path}/saved_model_simp.onnx 8135 400

Stopping http://0.0.0.0:8135
Serving '/data/nb_data/saved_models/saved_model_simp.onnx' at http://0.0.0.0:8135


In [102]:
for i, node in enumerate(model_simp.graph.node):
    if node.name == 'StatefulPartitionedCall/resnet_period_estimator/ExpandDims':
        print('%5d i', node.name)
        print(node)

%5d i StatefulPartitionedCall/resnet_period_estimator/ExpandDims
input: "StatefulPartitionedCall/resnet_period_estimator/Softmax:0"
input: "const_starts__835"
output: "StatefulPartitionedCall/resnet_period_estimator/ExpandDims:0"
name: "StatefulPartitionedCall/resnet_period_estimator/ExpandDims"
op_type: "Unsqueeze"



In [105]:
model_simp.graph.value_info

[name: "StatefulPartitionedCall/resnet_period_estimator/Reshape:0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 112
      }
      dim {
        dim_value: 112
      }
      dim {
        dim_value: 3
      }
    }
  }
}
, name: "StatefulPartitionedCall/resnet_period_estimator/model/conv1_conv/Conv2D__39:0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 112
      }
      dim {
        dim_value: 112
      }
    }
  }
}
, name: "Conv__910:0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 64
      }
      dim {
        dim_value: 56
      }
      dim {
        dim_value: 56
      }
    }
  }
}
, name: "StatefulPartitionedCall/resnet_period_estimator/model/pool1_pad/Pad:0"
type {
  tensor_type {
    elem_type: 1
    s

In [ ]:
for i, node in enumerate(model_simp.graph.initializer):
    print('%5d i', node.name)
    print(node)

### Repnet Inference

In [63]:
import cv2
from scipy.signal import medfilt
from scipy.special import softmax

def sigmoid(x):
    s = 1 / (1 + np.exp(-x))
    return s

num_frames = 64
# video_path = '/data/testvids/test1.mp4'
video_path = 'https://frepai.s3.didiyunapi.com/datasets/test/repnet_bird.gif'

def read_video(video_filename, width=112, height=112):
    cap = cv2.VideoCapture(video_filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = []
    if cap.isOpened():
        while True:
            success, frame_bgr = cap.read()
            if not success:
                break
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            frame_rgb = cv2.resize(frame_rgb, (width, height))
            frames.append(frame_rgb)
        frames = np.asarray(frames)
    return frames, fps

video_frames, fps = read_video(video_path)

In [40]:
# show_image(video_path)

In [41]:
seq_len = len(video_frames)
diff = math.ceil(seq_len / num_frames) * num_frames - seq_len
fps, seq_len, diff

(25.0, 260, 60)

In [98]:
# align num_frames
frames = np.take(video_frames, indices=list(range(seq_len)) + [seq_len-1] * diff, axis=0)
batch_size = int(len(frames) / num_frames)
len(frames), batch_size

(320, 5)

In [99]:
# preprocess image
inputs_numpy = np.reshape(frames, (batch_size, 64, 112, 112, 3)).astype(np.float32)
inputs_numpy -= 127.5
inputs_numpy /= 127.5

In [44]:
inputs_tensor = tf.convert_to_tensor(inputs_numpy)
outputs_tensor = model(inputs_tensor)

In [101]:
# inputs_numpy = np.random.randn(1, 64, 112, 112, 3).astype(np.float32)
sess = rt.InferenceSession(f'{saved_model_path}/saved_model_simp.onnx')  
inputs = {sess.get_inputs()[0].name: inputs_numpy}
outputs = [o.name for o in sess.get_outputs()]
outputs_onnx = sess.run(outputs, inputs)

In [86]:
outputs_tensor[1][0][:3].numpy().T, '-'*60, outputs_onnx[1][0][:3].T

(array([[0.75811225, 0.9941926 , 1.5130898 ]], dtype=float32),
 '------------------------------------------------------------',
 array([[0.7581339, 0.9942224, 1.5131185]], dtype=float32))

### Calculate Count

In [47]:
raw_scores1, within_period1, sim1 = outputs_tensor[0].numpy(), outputs_tensor[1].numpy(), outputs_tensor[2].numpy()
raw_scores2, within_period2, sim2 = outputs_onnx
(raw_scores1.shape, within_period1.shape, sim1.shape), '-'*50, (raw_scores2.shape, within_period2.shape, sim2.shape)

(((5, 64, 32), (5, 64, 1), (5, 64, 512)),
 '--------------------------------------------------',
 ((5, 64, 32), (5, 64, 1), (5, 64, 512)))

In [48]:
onnx_used = True
if not onnx_used:
    period_matrix = raw_scores1.reshape((-1, raw_scores1.shape[-1]))[:seq_len]
    within_period = within_period1.reshape((-1, within_period1.shape[-1]))[:seq_len]
else:
    period_matrix = raw_scores2.reshape((-1, raw_scores2.shape[-1]))[:seq_len]
    within_period = within_period2.reshape((-1, within_period2.shape[-1]))[:seq_len]
(period_matrix.shape, within_period.shape), '-'*60, \
period_matrix[0][:3], '-'*60, within_period[:3].T

(((260, 32), (260, 1)),
 '------------------------------------------------------------',
 array([-13.611417 ,   3.2236755,   1.8041031], dtype=float32),
 '------------------------------------------------------------',
 array([[0.7581339, 0.9942224, 1.5131185]], dtype=float32))

####  Inv-Period Confidence Score 

In [49]:
# Get best value from [3, 32] as the inv-period
per_frame_inv_period = np.argmax(period_matrix, axis=-1) + 1
per_frame_inv_period[-10:]

array([25, 25, 25, 25, 25, 25, 16, 16, 16, 16])

In [50]:
inv_period_conf = np.max(softmax(period_matrix, axis=-1), axis=-1)
inv_period_conf = np.where(np.less(per_frame_inv_period, 3), 0.0, inv_period_conf)
inv_period_conf[-10:]

array([0.7225712 , 0.7370858 , 0.58260685, 0.6355523 , 0.649649  ,
       0.6477884 , 0.16340947, 0.16876176, 0.16909385, 0.17123011],
      dtype=float32)

#### Within Period Confience Scores

In [51]:
within_period_conf = sigmoid(within_period)[:, 0]
within_period_conf[:10]

array([0.68094844, 0.7299211 , 0.8195229 , 0.89376783, 0.8785292 ,
       0.90211225, 0.93336856, 0.8507732 , 0.8894575 , 0.8985492 ],
      dtype=float32)

In [52]:
within_period_conf  = np.sqrt(np.multiply(within_period_conf, inv_period_conf))
within_period_conf[:10]

array([0.5663528 , 0.608796  , 0.68180966, 0.68128985, 0.7586823 ,
       0.751629  , 0.75716   , 0.7665566 , 0.7665918 , 0.7700015 ],
      dtype=float32)

#### Period Count

In [53]:
# median_filter
within_period_binary = np.asarray(within_period_conf > 0.5)
within_period_binary = medfilt(within_period_binary, 5)
within_period_binary[-10:]

array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [54]:
per_frame_counts = np.where(np.less(per_frame_inv_period, 3), 0.0, np.divide(1.0, per_frame_inv_period))
per_frame_counts = medfilt(per_frame_counts, 5)
per_frame_counts *= np.asarray(within_period_binary)
per_frame_counts[-10:]

array([0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.  , 0.  , 0.  , 0.  ])

In [55]:
sum(per_frame_counts)

10.055384615384567

## TFlite

In [56]:
loaded_model = tf.keras.models.load_model(saved_model_path)
loaded_model.summary()

Model: "resnet_period_estimator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, None, None, 1024)  5209600   
_________________________________________________________________
conv3d (Conv3D)              multiple                  14156288  
_________________________________________________________________
batch_normalization (BatchNo multiple                  2048      
_________________________________________________________________
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
dense (Dense)                multiple                  1049088   
_________________________________________________________________
dense_1 (Dense)              multiple                  1049088   
_________________________________________________________________
transformer_layer (Transform multiple      

In [57]:
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS, # tflite: some op not support
]
litemodel = converter.convert()
with open(f'{saved_model_path}/saved_model.tflite', 'wb') as f:
    f.write(litemodel)

INFO:tensorflow:Assets written to: /tmp/tmp3ytx6_g0/assets


INFO:tensorflow:Assets written to: /tmp/tmp3ytx6_g0/assets
/usr/local/lib/python3.6/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [58]:
interpreter = tf.lite.Interpreter(model_path=f'{saved_model_path}/saved_model.tflite')
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], test_inputs.shape) # modify input shape
interpreter.allocate_tensors()
input_details, output_details

([{'name': 'x',
   'index': 0,
   'shape': array([  1,  64, 112, 112,   3], dtype=int32),
   'shape_signature': array([ -1,  64, 112, 112,   3], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}],
 [{'name': 'Identity',
   'index': 443,
   'shape': array([ 1,  1, 32], dtype=int32),
   'shape_signature': array([-1, -1, 32], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}},
  {'name': 'Identity_1',
   'index': 567,
   'shape': array([1, 1, 1], dtype=int32),
   'shape_signature': array([-1, -1,  1], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': arra

In [59]:
interpreter.set_tensor(input_details[0]['index'], test_inputs)
interpreter.invoke()
output_data1 = interpreter.get_tensor(output_details[0]['index'])
output_data2 = interpreter.get_tensor(output_details[1]['index'])
output_data3 = interpreter.get_tensor(output_details[2]['index'])

In [60]:
output_data2[0][:10]

array([[0.84026015],
       [0.56363696],
       [0.97801125],
       [1.2081324 ],
       [0.85031843],
       [1.2657135 ],
       [1.1013507 ],
       [0.34697947],
       [1.3012416 ],
       [1.2406797 ]], dtype=float32)

## References

- https://github.com/onnx/onnx/blob/master/docs/Operators.md
- https://github.com/microsoft/onnxruntime/blob/master/docs/Versioning.md
- https://github.com/onnx/tensorflow-onnx/blob/master/support_status.md
- https://segmentfault.com/a/1190000039936376
- https://github.com/onnx/tensorflow-onnx/issues/1606
- https://github.com/onnx/tensorflow-onnx/issues/1634

## FAQs

[TensorFlow Lite: TensorListFromTensor, TensorListReserve, TensorListStack, While][1]

[1]: https://github.com/tensorflow/tensorflow/issues/33416